In [2]:
import os, re, sys, json, pickle, requests, psycopg2, numpy
from bs4 import BeautifulSoup
from unidecode import unidecode
from datetime import datetime
from dateutil.parser import parse

In [4]:
nyt_home_link = "https://www.nytimes.com/"
nyt_home_response = requests.get(nyt_home_link, timeout=5)
nyt_home_soup = BeautifulSoup(nyt_home_response.content, "html.parser")

In [9]:
nyt_section_base_href = 'https://nytimes.com/section/'
nyt_sections_pages = ['world', 'us', 'business', 'tech', 'science', 'health', 'sports']

In [10]:
def collect_nyt_article_hrefs(base, sections):
    article_hrefs = list()
    for section in sections:
        page = base + section
        response = requests.get(page, timeout=5)
        soup = BeautifulSoup(response.content, "html.parser")
        articles = soup.find_all('article')
        for article in articles:
            anchors = article.find_all('a')
            for a in anchors:
                if 'tips' not in a['href'] \
                    and 'interactive' not in a['href'] \
                    and 'slideshow' not in a['href'] \
                    and '/video/' not in a['href'] \
                    and 'nytimes.com' not in a['href'] \
                    and len(a['href']) > 2:
                        article_hrefs.append(a['href'])
    return article_hrefs

In [11]:
nyt_article_hrefs = collect_nyt_article_hrefs(nyt_section_base_href, nyt_sections_pages)
nyt_article_hrefs = numpy.unique(nyt_article_hrefs)

In [12]:
nyt_article_hrefs

array(['/2019/08/06/us/politics/congress-gun-control.html',
       '/2019/08/06/us/politics/trump-dayton-el-paso-shootings.html',
       '/2019/08/07/us/politics/cory-booker-speech-mother-emanuel.html',
       '/2019/08/07/us/politics/trump-el-paso-dayton-visits.html'],
      dtype='<U62')

In [497]:
def find_author(soup, href):
    date = '-'.join(href.split('/')[1:4])
    times = soup.find_all('time', {'datetime' : date})
    parents = [t.parent.parent.parent for t in times]
    author = ''
    for parent in parents:
        p = parent.find('p', {'itemprop' : 'author'})
        if p:
            author += p.text.replace('By ', '')
    return str(author)

In [498]:
def collect_paragraphs(soup):
    body = soup.find_all('section', {'name': 'articleBody'})
    paragraphs = list()
    for b in body:
        paragraphs.extend(b.find_all('p'))
    paragraphs = [str(unidecode(p)) for p in paragraphs]
    paragraphs = ''.join(paragraphs)
    return paragraphs

In [540]:
def parse_article_soup(soup, href):
    article = dict()
    entry = str(href).split('/')[-1]
    entry = entry.replace('.html', '')
    entry = entry.replace('#commentsContainer', '')
    article = dict()
    article['entry'] = entry
    article['meta'] = dict()
    article['meta']['title'] = unidecode(soup.find('title').text.replace(' - The New York Times', ''))
    article['meta']['author'] = find_author(soup, href)
    try:
        article['meta']['section'] = href.split('/')[4]
    except:
        pass
    article['meta']['href'] = "nytimes.com" + href
    article['meta']['date'] = "-".join(href.split('/')[1:4])
    article['meta']['path'] = str(href.split('/')[-1]).replace('#commentsContainer', '')
    article['article_body'] = collect_paragraphs(soup)
    return article

s = requests.session()
current_nyt_articles = dict()

for href in nyt_article_refs:
    response = s.get('https://nytimes.com/{0}'.format(href))
    soup = BeautifulSoup(response.content.decode('utf-8', 'ignore'), 'html.parser')
    article = parse_article_soup(href, soup)
    current_nyt_articles[article['entry']] = article
                                   

keys = list(current_nyt_articles.keys())
print(current_nyt_articles[keys[80]]['meta'])

In [500]:
def build_nyt_articles_dict(hrefs):
    current_nyt_articles = dict()
    base = 'https://nytimes.com/'
    
    for href in hrefs:
        response = requests.get(base + href)
        soup = BeautifulSoup(
            response.content.decode('utf-8', 'ignore'),
            'html.parser');
        article = parse_article_soup(soup, href)
        current_nyt_articles[article['entry']] = article
    
    return current_nyt_articles

In [501]:
def write_nyt_articles_files(articles_dict):
    nyt_articles_meta = dict()
    for key in articles_dict:
        article_file_path = key + '.html'
        nyt_articles_meta[key] = articles_dict[key]['meta']
        nyt_articles_meta[key]['file_path'] = article_file_path
        
        with open('../app/assets/articles/{}'.format(article_file_path), 'w+') as outfile:
            outfile.write(str(articles_dict[key]['article_body']))
    
    return nyt_articles_meta    

In [502]:
current_nyt_articles = build_nyt_articles_dict(nyt_article_hrefs)
current_nyt_articles

{'2020-candidates-election-2016': {'article_body': '<p class="css-exrw3m evys1bk0">WASHINGTON — Democrats still don’t agree on why they lost the White House in 2016. But they have absolutely no intention of repeating their mistakes.</p><p class="css-exrw3m evys1bk0">Whatever they were, exactly.</p><p class="css-exrw3m evys1bk0">Was it the loss of working-class white voters or urban black voters? Too much talk of Donald J. Trump or not enough? And what about those Russians?</p><p class="css-exrw3m evys1bk0">If Hillary Clinton’s decision to skip campaigning in Wisconsin was a problem, the party has that one covered. Democrats selected Milwaukee as the site of their national convention a year from now.</p><p class="css-exrw3m evys1bk0">In Michigan and Pennsylvania, states where Hillary Clinton lost after polling ahead for months, Democratic candidates pop up at coffee shops and farmers’ markets, field questions from audiences in packed high school auditoriums and clog up cellphone voice m

In [503]:
nyt_articles_meta = write_nyt_articles_files(current_nyt_articles)

#### Build Database

In [504]:
conn = psycopg2.connect(dbname='nytlitedb', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [541]:
create_table_sql = (
                'DROP TABLE IF EXISTS articles;'
                'CREATE TABLE articles ('
                'id SERIAL PRIMARY KEY,'
                'title text NOT NULL,'
                'author text NOT NULL,'
                'date text NOT NULL,'
                'section text NOT NULL,'
                'href text NOT NULL,'
                'path text NOT NULL);'
                )

In [506]:
cursor.execute(create_table_sql)
cursor.close()
conn.commit()

In [507]:
def create_insert_commands(nyt_articles_meta):
    keys = nyt_articles_meta.keys()
    for key in keys:
        article = nyt_articles_meta[key]
        insert_SQL = (
            "INSERT INTO articles "
            "(title, author, "
            "date, section, href, path) "
            "VALUES (%s, %s, %s, %s, %s, %s);")
        values = [article['title'], article['author'], article['date'], 
                      article['section'], article['href'], article['path']]
        yield (insert_SQL, values)

In [508]:
insert_commands = list(create_insert_commands(nyt_articles_meta))

In [509]:
print(insert_commands[2])

('INSERT INTO articles (title, author, date, section, href, path) VALUES (%s, %s, %s, %s, %s, %s);', ['Secret Venezuela Files Warn About Maduro Confidant', 'Nicholas Casey', '2019-05-02', 'world', '/2019/05/02/world/americas/venezuela-maduro-hezbollah-drugs.html', 'venezuela-maduro-hezbollah-drugs.html'])


In [510]:
def execute_inserts():
    conn = psycopg2.connect(dbname='nytlitedb', user='gqe', host='localhost', password='')
    cursor = conn.cursor()

    for insert_command in insert_commands:
        try:
            cursor.execute(insert_command[0], insert_command[1])
        except Exception as e:
            print(e)
        
    cursor.close()
    conn.commit()

In [537]:
delete_command = ( "DELETE FROM articles a USING articles b "
    "WHERE a.id > b.id "
    "AND a.title = b.title AND a.author = b.author;")

In [538]:
conn = psycopg2.connect(dbname='nytlitedb', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute(delete_command)

In [539]:
conn = psycopg2.connect(dbname='nytlitedb', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute('SELECT DISTINCT ON (title, author) * FROM articles')

rows = cursor.fetchall()
for row in rows[:600]:
    print(row)

(63, "'Space Is the New Black'", 'Vanessa Friedman', '2019-07-17', 'style', '/2019/07/17/style/space-is-the-new-black.html', 'space-is-the-new-black.html')
(78, '5 Million Bulgarians Have Their Personal Data Stolen in Hack', 'Marc Santora', '2019-07-17', 'world', '/2019/07/17/world/europe/bulgaria-hack-cyberattack.html', 'bulgaria-hack-cyberattack.html')
(36, 'A Better Way to Manage Your Period? Try the Menstrual Cup, Scientists Say', 'Roni Caryn Rabin', '2019-07-16', 'health', '/2019/07/16/health/menstrual-cup-periods-women.html', 'menstrual-cup-periods-women.html')
(68, "A Border Kept Him From His Daughter. Now He's Here to Say Goodbye.", 'Christina Goldbaum and Miriam Jordan', '2019-07-17', 'us', '/2019/07/17/us/migrant-teen-suicide-long-island.html', 'migrant-teen-suicide-long-island.html')
(25, "A Burning Lava Lake Concealed by a Volcano's Glacial Ice", 'Robin George Andrews', '2019-07-12', 'science', '/2019/07/12/science/lava-lake-volcano-antarctica.html', 'lava-lake-volcano-anta

In [11]:
conn = psycopg2.connect(dbname='nytlitedb', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute('SELECT * FROM articles')
rows = cursor.fetchall()
new_rows = list()

for i, row in enumerate(rows):
    today = datetime.now()
    datestr = datetime.strptime(row[3], '%Y-%m-%d')
    print(datestr - today)

-136 days, 13:44:47.337751
-124 days, 13:44:47.337570
-121 days, 13:44:47.337486
-106 days, 13:44:47.337422
-106 days, 13:44:47.337354
-106 days, 13:44:47.337052
-105 days, 13:44:47.336974
-100 days, 13:44:47.336910
-94 days, 13:44:47.336596
-85 days, 13:44:47.336342
-82 days, 13:44:47.336174
-78 days, 13:44:47.336088
-73 days, 13:44:47.336009
-64 days, 13:44:47.335930
-61 days, 13:44:47.335796
-61 days, 13:44:47.335726
-60 days, 13:44:47.335660
-58 days, 13:44:47.335533
-57 days, 13:44:47.335458
-55 days, 13:44:47.335313
-53 days, 13:44:47.335248
-51 days, 13:44:47.335187
-51 days, 13:44:47.335129
-51 days, 13:44:47.335063
-50 days, 13:44:47.334912
-49 days, 13:44:47.334856
-49 days, 13:44:47.334797
-49 days, 13:44:47.334742
-49 days, 13:44:47.334676
-48 days, 13:44:47.334519
-48 days, 13:44:47.334459
-47 days, 13:44:47.334403
-47 days, 13:44:47.334356
-47 days, 13:44:47.334264
-47 days, 13:44:47.334186
-46 days, 13:44:47.334096
-45 days, 13:44:47.334029
-46 days, 13:44:47.333970
-46 